In [10]:
# Load the model using joblib
import joblib
from collections import Counter
from helper.helper import format_number_and_round
from preprocessing.scarpping_component import extract_component_by_images
import os
import pandas as pd
import dlib
import cv2
from feature_extraction.poc import POC
from feature_extraction.vektor import Vektor
from feature_extraction.quadran import Quadran

# Load the model
model = joblib.load('models/knn_model_tanpa_hidung.joblib')
label_encoder = joblib.load('models/label_encoder_tanpa_hidung.joblib')

# Vidio directory
video_directory = 'dataset/CASME2/Sadness/Video_Image_RAW 200fps/17_EP15_01.avi' # Sadness
# video_directory = 'dataset\CASME2\Happiness\Compressed 30fps\S6_EP01_01.avi' # Happiness 

output_image_directory = 'output_image_5x5'

# Convert Video to Image
from preprocessing.input_to_image import get_frames_by_input_video

gambar = get_frames_by_input_video(video_directory, output_image_directory,framePerSecond=200)

components_setup = {
    'mulut': {
        'object_name': 'mouth',
        'object_rectangle': {"x_right": 54, "x_left": 48, "y_highest": 52, "y_lowest": 57},
        'pixel_shifting': {"pixel_x": 25, "pixel_y": 5},
        'object_dimension': {'width': 140, 'height': 40}
    },
    'mata_kiri': {
        'object_name': 'eye_left',
        'object_rectangle': {"x_right": 39, "x_left": 36, "y_highest": 38, "y_lowest": 41},
        'pixel_shifting': {"pixel_x": 25, "pixel_y": 25},
        'object_dimension': {'width': 90, 'height': 55}
    },
    'mata_kanan': {
        'object_name': 'eye_right',
        'object_rectangle': {"x_right": 45, "x_left": 42, "y_highest": 43, "y_lowest": 47},
        'pixel_shifting': {"pixel_x": 25, "pixel_y": 25},
        'object_dimension': {'width': 90, 'height': 55}
    },
    'alis_kiri': {
        'object_name': 'eyebrow_left',
        'object_rectangle': {"x_right": 21, "x_left": 17, "y_highest": 18, "y_lowest": 21},
        'pixel_shifting': {"pixel_x": 15, "pixel_y": 15},
        'object_dimension': {'width': 110, 'height': 40}
    },
    'alis_kanan': {
        'object_name': 'eyebrow_right',
        'object_rectangle': {"x_right": 26, "x_left": 22, "y_highest": 25, "y_lowest": 22},
        'pixel_shifting': {"pixel_x": 15, "pixel_y": 15},
        'object_dimension': {'width': 110, 'height': 40}
    },
    'hidung_kanan': {
        'object_name': 'nose_right',
        'object_rectangle': {"x_right": 31, "x_left": 40, "y_highest": 40, "y_lowest": 48},
        'pixel_shifting': {"pixel_x": 15, "pixel_y": -25},
        'object_dimension': {'width': 70, 'height': 40}
    },
    'hidung_kiri': {
        'object_name': 'nose_left',
        'object_rectangle': {"x_right": 47, "x_left": 35, "y_highest": 47, "y_lowest": 54},
        'pixel_shifting': {"pixel_x": 15, "pixel_y": -25},
        'object_dimension': {'width': 70, 'height': 40}
    }
}

blockSize = 5
frames_data_quadran = []
frames_data_all_component = []
frames_data_quadran_column = ['sumX', 'sumY', 'Tetha', 'Magnitude', 'JumlahQuadran']
quadran_dimensions = ['Q1', 'Q2', 'Q3', 'Q4']
frames_data = {component_name: [] for component_name in components_setup}
total_blocks_components = {component_name: 0 for component_name in components_setup}
data_blocks_first_image = {component_name: None for component_name in components_setup}
index = {component_name: 0 for component_name in components_setup}

# load model dan shape predictor untuk deteksi wajah
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("models/shape_predictor_68_face_landmarks.dat")

# mendapatkan hasil prediksi dari model
def get_calculate_from_predict(list_decoded_predictions):
    # Hitung jumlah kemunculan setiap kategori dalam array hasil prediksi
    prediction_counts = Counter(list_decoded_predictions)

    # Hitung total jumlah prediksi
    total_predictions = len(list_decoded_predictions)

    # Inisialisasi variabel untuk menyimpan kategori terbanyak (hasilnya) dan jumlahnya dan array list kategori
    result_prediction = None
    most_common_count = 0
    list_predictions = []

    # Lakukan iterasi melalui hasil prediksi
    for category, count in prediction_counts.items():
        # Hitung persentase dari setiap kategori
        percentage = (count / total_predictions) * 100

        # Tambahkan informasi jumlah dan persentase ke list
        list_predictions.append({
            "name": category,
            "count": count,
            "percentage": format_number_and_round(percentage, 2)
        })

        # Periksa apakah kategori saat ini memiliki jumlah terbanyak
        if count > most_common_count:
            most_common_count = count
            result_prediction = category

    return result_prediction, list_predictions

for filename in os.listdir(output_image_directory):
    if filename.endswith(".jpg") or filename.endswith(".png"): 
        image = cv2.imread(os.path.join(output_image_directory, filename))
        image = cv2.resize(image, (600, 500))
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        rects = detector(gray)

        if index[component_name] != 0:
            frame_data_all_component = {'Frame': f"{index[component_name] + 1}({filename.split('.')[0]})"}
            frame_data_quadran = {'Frame': f"{index[component_name] + 1}({filename.split('.')[0]})"}
                        
        for rect in rects:
            shape = predictor(gray, rect)
            for component_name, component_info in components_setup.items():
                sum_data_by_quadran = {}
                frame_data = {'Frame': f"{index[component_name] + 1}({filename.split('.')[0]})"}
                for column in frames_data_quadran_column:
                    sum_data_by_quadran[column] = {quadrant: 0 for quadrant in quadran_dimensions}

                data_blocks_image_current = extract_component_by_images(
                    image=image,
                    shape=shape,
                    frameName=filename.split(".")[0],
                    objectName=component_info['object_name'],
                    objectRectangle=component_info['object_rectangle'],
                    pixelShifting=component_info['pixel_shifting'],
                    objectDimension=component_info['object_dimension']
                )

                if data_blocks_first_image[component_name] is None:
                    frames_data[component_name].append(frame_data)
                    frame_data['Folder Path'] = "data_test"
                    frame_data['Label'] = "data_test"
                    data_blocks_first_image[component_name] = data_blocks_image_current
                    continue

                initPOC = POC(data_blocks_first_image[component_name], data_blocks_image_current, blockSize)
                valPOC = initPOC.getPOC() 

                initQuiv = Vektor(valPOC, blockSize)
                quivData = initQuiv.getVektor()

                initQuadran = Quadran(quivData) 
                quadran = initQuadran.getQuadran()

                for i, quad in enumerate(quadran):
                    # --- Setup bagian Nilai Fitur Dataset ---
                    # Set data kedalam frame_data sesuai column nya
                    frame_data[f'X{i+1}'] = quad[1]
                    frame_data[f'Y{i+1}'] = quad[2]
                    frame_data[f'Tetha{i+1}'] = quad[3]
                    frame_data[f'Magnitude{i+1}'] = quad[4]

                    # Set data kedalam frame_data_all_component sesuai columnnya
                    frame_data_all_component[f'{component_name}-X{i+1}'] = quad[1]
                    frame_data_all_component[f'{component_name}-Y{i+1}'] = quad[2]
                    frame_data_all_component[f'{component_name}-Tetha{i+1}'] = quad[3]
                    frame_data_all_component[f'{component_name}-Magnitude{i+1}'] = quad[4]

                    # --- Setup bagian 4qmv Dataset ---
                    # Cek apakah quad[5] ada didalam array quadran_dimensions
                    if quad[5] in quadran_dimensions:
                        # Tambahkan nilai quad[1] ke sumX pada kuadran yang sesuai
                        sum_data_by_quadran['sumX'][quad[5]] += quad[1]
                        # Tambahkan nilai quad[2] ke sumY pada kuadran yang sesuai
                        sum_data_by_quadran['sumY'][quad[5]] += quad[2]
                        # Tambahkan nilai quad[3] ke Tetha pada kuadran yang sesuai
                        sum_data_by_quadran['Tetha'][quad[5]] += quad[3]
                        # Tambahkan nilai quad[4] ke Magnitude pada kuadran yang sesuai
                        sum_data_by_quadran['Magnitude'][quad[5]] += quad[4]
                        # Tambahkan jumlah quadran sesuai dengan quad[5] ke JumlahQuadran pada kuadran yang sesuai
                        sum_data_by_quadran['JumlahQuadran'][quad[5]] += 1

                # frames_data[component_name].append(frame_data)

                frames_data[component_name].append(frame_data)
                # Tambahkan kolom "Folder Path" dengan nilai folder saat ini
                frame_data['Folder Path'] = "data_test"
                # Tambahkan kolom "Label" dengan nilai label saat ini
                frame_data['Label'] = "data_test"

                # --- Setup bagian 4qmv Dataset ---
                # Inisialisasi data untuk setiap blok dan setiap kuadran dengan nilai sesuai sum_data_by_quadran
                for quadran in quadran_dimensions:
                    for feature in frames_data_quadran_column:
                        # Buat nama kolom dengan menggunakan template yang diberikan
                        column_name = f"{component_name}{feature}{quadran}"
                        # Set value sum_data_by_quadran[feature][quadran] ke frame_data_quadran sesuai column_name nya
                        frame_data_quadran[column_name] = sum_data_by_quadran[feature][quadran]

        if not index[component_name] == 0:
            # --- Setup bagian 4qmv Dataset ---
            # Append data frame ke list frames_data_quadran untuk 4qmv
            frames_data_quadran.append(frame_data_quadran)
            # Tambahkan kolom "Folder Path" dengan nilai folder saat ini
            frame_data_quadran['Folder Path'] = "data_test"
            # Tambahkan kolom "Label" dengan nilai label saat ini
            frame_data_quadran['Label'] = "data_test"

            # --- Setup bagian frames data all component Dataset ---
            # Append data frame ke list frames_data_quadran untuk 4qmv
            frames_data_all_component.append(frame_data_all_component)
            # Tambahkan kolom "Folder Path" dengan nilai folder saat ini
            frame_data_all_component['Folder Path'] = "data_test"
            # Tambahkan kolom "Label" dengan nilai label saat ini
            frame_data_all_component['Label'] = "data_test"

        # Update index per component_name
        index[component_name] += 1

df_fitur_all = pd.DataFrame(frames_data_all_component)
except_feature_columns = ['Frame', 'Folder Path', 'Label']  

df_fitur_all = df_fitur_all.drop(columns=except_feature_columns)

# force :3888 
df_fitur_all = df_fitur_all.iloc[:, :3888]

# Lakukan prediksi dengan model yang telah dimuat
predictions = model.predict(df_fitur_all.values)

# Ubah prediksi numerik menjadi label asli
decoded_predictions = label_encoder.inverse_transform(predictions)

# Hitung jumlah kemunculan setiap kategori dalam array hasil prediksi
prediction_counts = Counter(decoded_predictions)

# Hitung total jumlah prediksi
total_predictions = len(decoded_predictions)

# Inisialisasi variabel untuk menyimpan kategori terbanyak (hasilnya) dan jumlahnya
result_prediction = None
most_common_count = 0

# Lakukan iterasi melalui hasil prediksi untuk mencari kategori dengan frekuensi tertinggi
for category, count in prediction_counts.items():
    # Hitung persentase dari setiap kategori
    percentage = (count / total_predictions) * 100

    # Print informasi tentang setiap kategori
    print(f"Kategori: {category}, Jumlah: {count}, Persentase: {percentage:.2f}%")

    # Periksa apakah kategori saat ini memiliki jumlah terbanyak
    if count > most_common_count:
        most_common_count = count
        result_prediction = category

# Print hasil prediksi berdasarkan kategori yang paling sering muncul
print(f"\nHasil Prediksi: {result_prediction}, Jumlah Terbanyak: {most_common_count}, Persentase: {(most_common_count / total_predictions) * 100:.2f}%")

width_object: 140, height_object: 40
width_object: 90, height_object: 55
width_object: 90, height_object: 55
width_object: 110, height_object: 40
width_object: 110, height_object: 40
width_object: 70, height_object: 40
width_object: 70, height_object: 40
width_object: 140, height_object: 40
width_object: 90, height_object: 55
width_object: 90, height_object: 55
width_object: 110, height_object: 40
width_object: 110, height_object: 40
width_object: 70, height_object: 40
width_object: 70, height_object: 40
width_object: 140, height_object: 40
width_object: 90, height_object: 55
width_object: 90, height_object: 55
width_object: 110, height_object: 40
width_object: 110, height_object: 40
width_object: 70, height_object: 40
width_object: 70, height_object: 40
width_object: 140, height_object: 40
width_object: 90, height_object: 55
width_object: 90, height_object: 55
width_object: 110, height_object: 40
width_object: 110, height_object: 40
width_object: 70, height_object: 40
width_object: 70